In [1]:
from gurobipy import *
import numpy as np
import string
import random

## F1-Score

In [12]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
C = sum(S)

xe = range(1, 11)
ye = range(1, 11)

dic = list(string.ascii_lowercase)
E2words = {}

for i in range(10):
    E2words[i] = set(random.sample(dic, 5))

Email, S, t, xe, ye, words= multidict({i: [S[i], i, float(xe[i]), float(ye[i]), E2words[i]] for i in range(10)})

f1_t = (2*C/len(S))/(1+C/len(S))
print(xe)
print(ye)
print(E2words)
print(t)


{0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0, 5: 6.0, 6: 7.0, 7: 8.0, 8: 9.0, 9: 10.0}
{0: 1.0, 1: 2.0, 2: 3.0, 3: 4.0, 4: 5.0, 5: 6.0, 6: 7.0, 7: 8.0, 8: 9.0, 9: 10.0}
{0: set(['g', 'r', 'm', 'd', 'o']), 1: set(['a', 'p', 'c', 'u', 'o']), 2: set(['q', 'e', 'j', 'm', 'f']), 3: set(['a', 'e', 'c', 'b', 'm']), 4: set(['x', 'k', 'o', 'w', 'h']), 5: set(['y', 'j', 't', 'g', 'f']), 6: set(['p', 'c', 'z', 'l', 'o']), 7: set(['a', 'q', 'd', 'g', 'n']), 8: set(['r', 'z', 'e', 'd', 'g']), 9: set(['m', 'u', 'd', 'o', 'f'])}
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}


In [31]:
model = Model('F1')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
Iemail = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'Indication')
u = model.addVar(lb=0.0, ub =1.0, vtype='C', name = 'Sum_Indication')

# time field
#It = model.addVars(Email, vtype='B', name = 'time_indication')

Is = model.addVars(Email, vtype='B', name = 'Is-Var')
Ie = model.addVars(Email, vtype='B', name = 'Ie-Var')

ts = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-Start')
te = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-End')

# keyword field
Iw = model.addVars(dic, vtype='B', name = 'word_Indication')

# position field
#Ip = model.addVars(Email, vtype='B', name = 'posiition_indication')

Ixs = model.addVars(Email, vtype='B', name = 'Ixs-Var')
Ixl = model.addVars(Email, vtype='B', name = 'Ixl-Var')
Iys = model.addVars(Email, vtype='B', name = 'Iys-Var')
Iyl = model.addVars(Email, vtype='B', name = 'Iyl-Var')

xmin = model.addVar(lb=xe[0], ub=xe[9], vtype='I', name = 'Xmin')
xmax = model.addVar(lb=xe[0], ub=xe[9], vtype='I', name = 'Xmax')
ymin = model.addVar(lb=ye[0], ub=ye[9], vtype='I', name = 'Ymin')
ymax = model.addVar(lb=ye[0], ub=ye[9], vtype='I', name = 'Ymax')



In [32]:
model.addConstr(quicksum(w[i] for i in range(10))+u*C == 1.0, 'sum(w)+uC=1')

for i in Email:
    model.addConstr(w[i] <= u, "w["+str(i)+"]<=u")
    model.addConstr(w[i] <= 1e4*Ie[i], "w["+str(i)+"]<=M*I("+str(i)+")")
    model.addConstr(w[i] >= u-1e4*(1-Ie[i]),"w["+str(i)+"]>=u-M*[1-I("+str(i)+")]")
    model.addConstr(w[i] >= 0.0, "Weight-positive["+str(i)+"]")
    
    #model.addGenConstrAnd(Iemail[i], [It[i], Ip[i]], "Time_position_Con")
    
    #Time field
    model.addGenConstrAnd(Iemail[i], [Is[i], Ie[i]], "TimeCon["+str(i)+"]")
    
    model.addConstr(ts <= te, 'start<=end')
    
    model.addGenConstrIndicator(Is[i], True, ts-t[i] <= 0)
    model.addGenConstrIndicator(Ie[i], True, t[i]-te <= 0)
    
    model.addGenConstrIndicator(Is[i], False, ts-t[i] >= 1)
    model.addGenConstrIndicator(Ie[i], False, t[i]-te >= 1)
    
    #keyword field
    model.addGenConstrAnd(Iemail[i], [Iw[word] for word in E2words[i]], "keyword_Email_Con["+str(i)+"]")
    
    #Position field
    model.addGenConstrAnd(Iemail[i], [Ixs[i], Ixl[i], Iys[i], Iyl[i]], "PosCon["+str(i)+"]")
    
    model.addConstr(xmin <= xmax, 'xmin<=xmax')
    model.addConstr(ymin <= ymax, 'ymin<=xmax')
    
    model.addGenConstrIndicator(Ixs[i], True, xmin-xe[i] <= 0.0)
    model.addGenConstrIndicator(Ixl[i], True, xe[i]-xmax <= 0.0)
    model.addGenConstrIndicator(Iys[i], True, ymin-ye[i] <= 0.0)
    model.addGenConstrIndicator(Iyl[i], True, ye[i]-ymax <= 0.0)
    
    model.addGenConstrIndicator(Ixs[i], False, xmin-xe[i] >= 1)
    model.addGenConstrIndicator(Ixl[i], False, xe[i]-xmax >= 1)
    model.addGenConstrIndicator(Iys[i], False, ymin-ye[i] >= 1)
    model.addGenConstrIndicator(Iyl[i], False, ye[i]-ymax >= 1)
    
    
        
model.setObjective(w.prod(S), GRB.MAXIMIZE)
    
model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal*2)

print(f1_t)
model.write("f1-score.lp")


Optimize a model with 71 rows, 113 columns and 151 nonzeros
Model has 150 general constraints
Variable types: 11 continuous, 102 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+04]
Presolve added 184 rows and 0 columns
Presolve removed 0 rows and 21 columns
Presolve time: 0.03s
Presolved: 255 rows, 92 columns, 802 nonzeros
Variable types: 10 continuous, 82 integer (76 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 0.3333333
Found heuristic solution: objective 0.3846154

Root relaxation: objective 4.545455e-01, 120 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.45455    0    9    0.38462    0.45455  18.2%     -    0s
H    0     0                       0.4000000    0.45455 

## Precision LP

In [31]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]

xe = range(1, 11)
ye = range(1, 11)

dic = list(string.ascii_lowercase)
E2words = {}

for i in range(10):
    E2words[i] = set(random.sample(dic, 5))

Email, S, t, xe, ye, words= multidict({i: [S[i], i, float(xe[i]), float(ye[i]), E2words[i]] for i in range(10)})

In [32]:
model = Model('Precision')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
Iemail = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'email_Indication')
u = model.addVar(lb=0.0, ub =1.0, vtype='C', name = 'Sum_Indication')

# time field
#It = model.addVars(Email, vtype='B', name = 'time_indication')

Is = model.addVars(Email, vtype='B', name = 'Is-Var')
Ie = model.addVars(Email, vtype='B', name = 'Ie-Var')

ts = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-Start')
te = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-End')

# position field
#Ip = model.addVars(Email, vtype='B', name = 'posiition_indication')

Ixs = model.addVars(Email, vtype='B', name = 'Ixs-Var')
Ixl = model.addVars(Email, vtype='B', name = 'Ixl-Var')
Iys = model.addVars(Email, vtype='B', name = 'Iys-Var')
Iyl = model.addVars(Email, vtype='B', name = 'Iyl-Var')

xmin = model.addVar(lb=xe[0], ub=xe[9], vtype='I', name = 'Xmin')
xmax = model.addVar(lb=xe[0], ub=xe[9], vtype='I', name = 'Xmax')
ymin = model.addVar(lb=ye[0], ub=ye[9], vtype='I', name = 'Ymin')
ymax = model.addVar(lb=ye[0], ub=ye[9], vtype='I', name = 'Ymax')

# keyword field
Iw = model.addVars(dic, vtype='B', name = 'word_Indication')

In [33]:
model.addConstr(quicksum(w[i] for i in range(10)) == 1.0, 'sum(w)+uC=1')

for i in Email:
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*Ie[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-Ie[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    #model.addGenConstrAnd(Iemail[i], [It[i], Ip[i]], "Time_position_Con")
    
    #Time field
    model.addGenConstrAnd(Iemail[i], [Is[i], Ie[i]], "TimeCon")
    
    model.addConstr(ts <= te, 'start <= end')
    
    model.addGenConstrIndicator(Is[i], True, ts-t[i] <= 0)
    model.addGenConstrIndicator(Ie[i], True, t[i]-te <= 0)
    
    model.addGenConstrIndicator(Is[i], False, ts-t[i] >= 1)
    model.addGenConstrIndicator(Ie[i], False, t[i]-te >= 1)
    
    #Position field
    model.addGenConstrAnd(Iemail[i], [Ixs[i], Ixl[i], Iys[i], Iyl[i]], "PosCon")
    
    model.addConstr(xmin <= xmax, 'xmin <= xmax')
    model.addConstr(ymin <= ymax, 'ymin <= xmax')
    
    model.addGenConstrIndicator(Ixs[i], True, xmin-xe[i] <= 0.0)
    model.addGenConstrIndicator(Ixl[i], True, xe[i]-xmax <= 0.0)
    model.addGenConstrIndicator(Iys[i], True, ymin-ye[i] <= 0.0)
    model.addGenConstrIndicator(Iyl[i], True, ye[i]-ymax <= 0.0)
    
    model.addGenConstrIndicator(Ixs[i], False, xmin-xe[i] >= 1)
    model.addGenConstrIndicator(Ixl[i], False, xe[i]-xmax >= 1)
    model.addGenConstrIndicator(Iys[i], False, ymin-ye[i] >= 1)
    model.addGenConstrIndicator(Iyl[i], False, ye[i]-ymax >= 1)
    
    #keyword field
    model.addGenConstrOr(Iemail[i], [Iw[word] for word in E2words[i]], "keyword_Email_Con")
        
model.setObjective(w.prod(S), GRB.MAXIMIZE)
    
model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal)

Optimize a model with 71 rows, 113 columns and 150 nonzeros
Model has 150 general constraints
Variable types: 11 continuous, 102 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+04]
Presolve removed 92 rows and 42 columns
Presolve time: 0.02s
Presolved: 299 rows, 251 columns, 661 nonzeros
Presolved model has 108 SOS constraint(s)
Variable types: 118 continuous, 133 integer (127 binary)
Found heuristic solution: objective 0.5000000
Found heuristic solution: objective 0.8000000

Root relaxation: objective 8.333333e-01, 119 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.83333    0   43    0.80000    0.83333  4.17%     -    0s
     0     0    0.83333    0   29    0.80000    0.83333  4.17%     -    0s
     0     0    0.8

## Recall LP

In [34]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
u = 1/sum(S)

xe = range(1, 11)
ye = range(1, 11)

dic = list(string.ascii_lowercase)
E2words = {}

for i in range(10):
    E2words[i] = set(random.sample(dic, 5))

Email, S, t, xe, ye, words= multidict({i: [S[i], i, float(xe[i]), float(ye[i]), E2words[i]] for i in range(10)})

In [35]:
model = Model('Precision')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
Iemail = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'email_Indication')

# time field
#It = model.addVars(Email, vtype='B', name = 'time_indication')

Is = model.addVars(Email, vtype='B', name = 'Is-Var')
Ie = model.addVars(Email, vtype='B', name = 'Ie-Var')

ts = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-Start')
te = model.addVar(lb=t[0], ub=t[9], vtype='I', name = 'time-End')

# position field
#Ip = model.addVars(Email, vtype='B', name = 'posiition_indication')

Ixs = model.addVars(Email, vtype='B', name = 'Ixs-Var')
Ixl = model.addVars(Email, vtype='B', name = 'Ixl-Var')
Iys = model.addVars(Email, vtype='B', name = 'Iys-Var')
Iyl = model.addVars(Email, vtype='B', name = 'Iyl-Var')

xmin = model.addVar(lb=xe[0], ub=xe[9], vtype='I', name = 'Xmin')
xmax = model.addVar(lb=xe[0], ub=xe[9], vtype='I', name = 'Xmax')
ymin = model.addVar(lb=ye[0], ub=ye[9], vtype='I', name = 'Ymin')
ymax = model.addVar(lb=ye[0], ub=ye[9], vtype='I', name = 'Ymax')

# keyword field
Iw = model.addVars(dic, vtype='B', name = 'word_Indication')

In [36]:
for i in Email:
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*Ie[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-Ie[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    #model.addGenConstrAnd(Iemail[i], [It[i], Ip[i]], "Time_position_Con")
    
    #Time field
    model.addGenConstrAnd(Iemail[i], [Is[i], Ie[i]], "TimeCon")
    
    model.addConstr(ts <= te, 'start <= end')
    
    model.addGenConstrIndicator(Is[i], True, ts-t[i] <= 0)
    model.addGenConstrIndicator(Ie[i], True, t[i]-te <= 0)
    
    model.addGenConstrIndicator(Is[i], False, ts-t[i] >= 1)
    model.addGenConstrIndicator(Ie[i], False, t[i]-te >= 1)
    
    #Position field
    model.addGenConstrAnd(Iemail[i], [Ixs[i], Ixl[i], Iys[i], Iyl[i]], "PosCon")
    
    model.addConstr(xmin <= xmax, 'xmin <= xmax')
    model.addConstr(ymin <= ymax, 'ymin <= xmax')
    
    model.addGenConstrIndicator(Ixs[i], True, xmin-xe[i] <= 0.0)
    model.addGenConstrIndicator(Ixl[i], True, xe[i]-xmax <= 0.0)
    model.addGenConstrIndicator(Iys[i], True, ymin-ye[i] <= 0.0)
    model.addGenConstrIndicator(Iyl[i], True, ye[i]-ymax <= 0.0)
    
    model.addGenConstrIndicator(Ixs[i], False, xmin-xe[i] >= 1)
    model.addGenConstrIndicator(Ixl[i], False, xe[i]-xmax >= 1)
    model.addGenConstrIndicator(Iys[i], False, ymin-ye[i] >= 1)
    model.addGenConstrIndicator(Iyl[i], False, ye[i]-ymax >= 1)
    
    #keyword field
    model.addGenConstrOr(Iemail[i], [Iw[word] for word in E2words[i]], "keyword_Email_Con")
        
model.setObjective(w.prod(S), GRB.MAXIMIZE)
    
model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal)

Optimize a model with 70 rows, 112 columns and 120 nonzeros
Model has 150 general constraints
Variable types: 10 continuous, 102 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [2e-01, 1e+04]
Presolve removed 109 rows and 48 columns
Presolve time: 0.00s
Presolved: 281 rows, 244 columns, 616 nonzeros
Presolved model has 108 SOS constraint(s)
Variable types: 108 continuous, 136 integer (130 binary)

Root relaxation: objective 1.000000e+00, 105 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0   31          -    1.00000      -     -    0s
     0     2    1.00000    0   31          -    1.00000      -     -    0s
*  263     9              16       1.0000000    1.00000  0.00%   2.8    0s

Cutting planes:
  Implied bound: 1